In [3]:
# Create a multiple dispatch decorator
registry = {}

class MultiMethod(object):
    
    def __init__(self, name):
        self.name = name
        self.typemap = {}
        
    def __call__(self, *args):
        # Extract class from arguments
        types = tuple(arg.__class__ for arg in args) # a generator expression!
        # Use class tuple as lookup key
        # TODO: Allow subclasses to match as well
        function = self.typemap.get(types)
        if function is None:
            raise TypeError("no match")
        return function(*args)
    
    def register(self, types, function):
        if types in self.typemap:
            raise TypeError("duplicate registration")
        # Map argument class types to this function
        self.typemap[types] = function


def multimethod(*types):
    def register(function):
        # Check if the function we're wrapping was already wrapped
        function = getattr(function, "__lastreg__", function)
        name = function.__name__
        
        mm = registry.get(name)
        # Create multimethod registry, if needed
        if mm is None:
            mm = registry[name] = MultiMethod(name)
            
        # Register this function
        mm.register(types, function)
        # Save a reference to this function, in case there's another
        # multimethod decorator above us
        mm.__lastreg__ = function
        return mm
    return register